<img src="./logo_UNSAM.jpg" align="right" width="150" /> 

# TS5: Estimación espectral: Ancho de banda de señales reales 

**Alumnas:** María Victoria Poric y Agustina Rocío Paolini Rosso  
**Carrera:** Ingeniería Biomédica    
**Materia:** Análisis y Procesamiento de Señales    
**Fecha:** Octubre 2025 

## Consigna

Tomar del repositorio `PDStestbench` los siguientes tres tipos de señales registradas:

- Electrocardiograma (ECG): `ECG_TP4.mat` — Registro electrocardiográfico durante una prueba de esfuerzo.  
- Pletismografía (PPG): `PPG.csv` — Señal registrada en reposo.  
- Audio: Tres registros con voz y dos registros con silbidos.

1) Estimar la densidad espectral de potencia (PSD) de cada señal utilizando alguno de los métodos vistos en clase:
- Periodograma ventaneado  
- Welch  
- Blackman–Tukey

2) A partir de la PSD:
- Estimar el ancho de banda de cada señal.  
- Presentar los resultados en una tabla comparativa.

*Bonus*:
- Proponer y analizar una señal adicional, explicando su origen y cómo fue digitalizada. Repetir el análisis de PSD y ancho de banda.

## Introducción
El análisis espectral es una herramienta esencial en el procesamiento de señales, ya que permite estudiar cómo se distribuye la energía de una señal en el dominio de la frecuencia. En muchos casos, como en señales biomédicas o acústicas, la observación directa en el dominio temporal no es suficiente para comprender los fenómenos que intervienen en la generación de la señal. Por ejemplo, en un electrocardiograma (ECG) o una señal de pletismografía (PPG), los componentes de baja frecuencia están asociados a procesos fisiológicos lentos, mientras que las componentes de mayor frecuencia pueden corresponder a ruido o artefactos. En señales de audio, por otra parte, el contenido espectral define la calidad y las características tonales del sonido.

Para caracterizar cuantitativamente este contenido frecuencial se utiliza la Densidad Espectral de Potencia (Power Spectral Density, PSD), la cual describe cómo se distribuye la potencia de la señal en función de la frecuencia. En otras palabras, la PSD permite identificar qué frecuencias contienen mayor energía, ofreciendo una visión más profunda de la dinámica interna de la señal.

Matemáticamente, para una señal estacionaria $x(t)$ con transformada de Fourier $X(f)$, la PSD se define como:

$$
S_{xx}(f) = \lim_{T \to \infty} \frac{1}{T} \, E\left\{ \left| X_T(f) \right|^2 \right\} \tag{1}
$$

donde $X_T(f)$ es la transformada de Fourier de la señal truncada a un intervalo de duración $T$, y $E{⋅}$ denota el valor esperado. Sin embargo, en la práctica trabajamos con señales discretas y de duración finita, por lo que es necesario estimar la PSD a partir de los datos disponibles.

El **método de Welch** es una de las técnicas más utilizadas para estimar la PSD de manera robusta. Surge como una mejora del periodograma clásico, que aunque conceptualmente simple, presenta alta variabilidad en sus estimaciones. Welch propone reducir dicha varianza mediante un proceso de promediado:

1) *Segmentación*: la señal $x[n]$ se divide en varios segmentos solapados de longitud $L$.
   
2) *Ventaneo*: cada segmento se multiplica por una ventana (por ejemplo, Hamming o Hann) para minimizar las fugas espectrales causadas por la truncación.

3) *Cálculo del periodograma*: se obtiene la potencia espectral de cada segmento:
$$
P_k(f) = \frac{1}{L U} \left| \sum_{n=0}^{L-1} w[n] \, x_k[n] \, e^{-j 2 \pi f n} \right|^2 \tag{2}
$$
donde $w[n]$ es la ventana aplicada, $U$ un factor de normalización, y $x_k[n]$ es el segmento k-ésimo de la señal.  

4) *Promediado*: los periodogramas de todos los segmentos se promedian:
$$
S_{xx}(f) = \frac{1}{K} \sum_{k=1}^{K} P_k(f) \tag{3}
$$
obteniendo así una estimación más suave y confiable de la densidad espectral.

Este procedimiento mejora la estabilidad del resultado a costa de una ligera pérdida en la resolución en frecuencia, generando un equilibrio adecuado entre precisión y robustez.

El método de Welch resulta particularmente apropiado para señales fisiológicas y de audio, ya que estas suelen ser cuasi-estacionarias (su estadística cambia lentamente en el tiempo), y el promediado permite reducir los efectos del ruido y las variaciones transitorias.

A partir de la PSD estimada, es posible calcular el **ancho de banda efectivo** de cada señal, es decir, el rango de frecuencias donde se concentra la mayor parte de su energía. Este parámetro proporciona información relevante sobre la naturaleza de la señal y sobre los requisitos de muestreo y filtrado necesarios para su procesamiento.

Una forma común de estimarlo es identificar el intervalo de frecuencias $[f1,f2]$ que contiene un porcentaje significativo (por ejemplo, el 95%) de la potencia total:

$$
\int_{f_1}^{f_2} S_{xx}(f) \, df = 0.95 \int_{0}^{f_{\text{max}}} S_{xx}(f) \, df \tag{4}
$$

Comparar el ancho de banda entre señales de distinta naturaleza (ECG, PPG y audio) permite comprender sus diferencias fundamentales: mientras que las señales fisiológicas suelen concentrar su energía en frecuencias bajas, las señales acústicas presentan una distribución más amplia y rica en armónicos.

El objetivo principal de este trabajo es estimar y analizar la densidad espectral de potencia (PSD) de distintos tipos de señales, como la del electrocardiograma (ECG), pletismografía (PPG) y audio, utilizando el método de Welch como técnica de estimación. A partir de las PSD obtenidas, se busca determinar el ancho de banda característico de cada señal y comparar sus comportamientos espectrales, resaltando las diferencias entre señales fisiológicas y acústicas. De esta manera, el estudio permite no solo aplicar herramientas de procesamiento digital de señales vistas en clase, sino también interpretar los resultados desde un punto de vista práctico y fisiológico, comprendiendo cómo el contenido en frecuencia refleja las propiedades y el origen físico de cada señal.

---
## Análisis y desarrollo
En primer lugar, se cargaron tres tipos de señales:

- ECG (Electrocardiograma) con una frecuencia de muestreo de `fs = 1000 Hz`,

- PPG (Pletismografía) con una frecuencia de muestreo de `fs = 400 Hz`,

- Audio proveniente de un archivo .wav, del cual también se obtuvo automáticamente su frecuencia de muestreo.

In [1]:
import numpy as np
from scipy import signal as sig
import matplotlib.pyplot as plt
import scipy.io as sio

# =======================================================
# CONFIGURACIÓN Y LECTURA DE SEÑALES
# =======================================================
# --- ECG sin ruido ---
fs_ecg = 1000  # Hz
ecg_sin_ruido = np.load('ecg_sin_ruido.npy').flatten()

# --- PPG sin ruido ---
fs_ppg = 400  # Hz
ppg = np.load('ppg_sin_ruido.npy').flatten()

# --- Audio ---
fs_audio, wav_data = sio.wavfile.read('la cucaracha.wav')
wav_data = wav_data.astype(float)


FileNotFoundError: [Errno 2] No such file or directory: 'la cucaracha.wav'

Estas señales representan distintos dominios de aplicación: el ECG y la PPG corresponden a señales biomédicas, que contienen información sobre la actividad cardíaca y hemodinámica, respectivamente. El análisis espectral en estos casos permite identificar las frecuencias dominantes asociadas a los ritmos fisiológicos (por ejemplo, la frecuencia cardíaca y sus armónicos), así como discriminar posibles ruidos.

En el caso de la señal de audio, el análisis de la densidad espectral de potencia (PSD) es especialmente útil para caracterizar el contenido armónico y tonal, ya que la energía de la señal se distribuye principalmente en bandas de frecuencia que corresponden a los fonemas, la melodía y el timbre. 

Contar con la frecuencia de muestreo de cada señal es fundamental para realizar un análisis espectral correcto, ya que esta determina el rango máximo de frecuencias representables (según el teorema de Nyquist).

Luego, se definió la función para obtener la estimación de la PSD y el ancho de banda de cada señal:

In [ ]:
# =======================================================
# FUNCIÓN GENERAL DE PSD Y ANCHO DE BANDA
# =======================================================
def calcular_psd_y_bw(signal, fs, n_promedios, nfft_factor, color, nombre, fmax=None, mostrar_min=False):

    nperseg = signal.shape[0] // n_promedios
    nfft = nfft_factor * nperseg

    f, Pxx = sig.welch(signal, fs=fs, window='hamming', nperseg=nperseg, nfft=nfft)
    
    df = f[1] - f[0]
    pot_total = np.sum(Pxx) * df
    pot_acumulada = np.cumsum(Pxx) * df

    # índices del 1% y 99% de energía acumulada
    indice_99 = np.where(pot_acumulada >= 0.99 * pot_total)[0][0]
    bw_max = f[indice_99]

    if mostrar_min:
        indice_01 = np.where(pot_acumulada >= 0.01 * pot_total)[0][0]
        bw_min = f[indice_01]
    else:
        bw_min = None

    # Gráfico PSD
    plt.figure()
    plt.plot(f, Pxx, color=color)
    if mostrar_min:
        plt.axvline(bw_min, color=color, linestyle='--', alpha=0.6)
        plt.axvline(bw_max, color=color, linestyle='--', alpha=0.6,
                    label=f'BW = {bw_max - bw_min:.1f} Hz')
    else:
        plt.axvline(bw_max, color=color, linestyle='--',
                    label=f'BW = {bw_max:.2f} Hz')

    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Densidad espectral')
    plt.title(f'Densidad espectral de potencia - {nombre}')
    plt.grid(True)
    if fmax:
        plt.xlim(0, fmax)
    plt.legend()
    plt.tight_layout()
    plt.show()

    if mostrar_min:
        return bw_min, bw_max, bw_max - bw_min
    else:
        return bw_max

El objetivo de la función `calcular_psd_y_bw` es calcular la densidad espectral de potencia (PSD) de una señal mediante el método de Welch, estimar su ancho de banda que contiene el 99 % de la potencia, y graficar la PSD con las frecuencias límites destacadas.

En primer lugar, se determina `nperseg` que corresponde a la longitud de cada segmento para Welch. Al utilizar `signal.shape[0]//n_promedios`, se divide la señal en *n_promedios* bloques, lo que permite promediar periodogramas y reducir la varianza de la estimación. Luego, se calcula `nfft` que establece el número de puntos de la FFT. Si `nfft > nperseg`, se realiza zero-padding, lo que incrementa la resolución en frecuencia sin añadir nueva información. 

A continuación, se calcula la PSD mediante `sig.welch`, donde `f` contiene las frecuencias correspondientes a cada bin de la PSD, y `Pxx` es la densidad espectral estimada.

Se utiliza la ventana Hamming para minimizar la fuga espectral, ya que la señal es truncada por segmentos. La ventana reduce la distorsión causada por discontinuidades en los extremos de cada segmento, suavizando los bordes y mejorando la estimación de la PSD. Asimismo, se calcula el ancho de cada bin en Hz denotado como *df*. 

Mediante una aproximación discreta de la integral de la PSD se calcula la potencia total aproximada:

$$
\text{pot\_total} \approx \int_0^{f_\text{max}} S_{xx}(f) \, df \approx \sum_{i} P_{xx}[i] \, \Delta f
$$

Y, además, se obtiene la potencia acumulada por frecuencia que es aproximadamente la energía total desde `0` hasta `f[i]`, es decir, el acumulado discreto de la PSD.

$$
\text{pot\_acumulada}[i] \approx \int_0^{f[i]} S_{xx}(f) \, df \approx \sum_{j=0}^{i} P_{xx}[j] \, \Delta f
$$

Para determinar el ancho de banda efectivo, se identifica la frecuencia donde se alcanza el 99 % de la potencia acumulada. Este valor se utiliza porque permite capturar prácticamente toda la energía de la señal sin verse afectada por componentes de muy baja energía o ruido de fondo que podrían alargar indefinidamente el rango de frecuencias. En otras palabras, el 99 % representa un criterio práctico que incluye la mayor parte de la información significativa de la señal, descartando contribuciones insignificantes. Opcionalmente, si `mostrar_min = True`, también se calcula el ancho de banda inferior correspondiente al 1 % de la potencia. El ancho de banda efectivo de la señal se define como:

$$
\text{BW} = f_\text{max\_potencia} - f_\text{min\_potencia}
$$
donde $f_\text{max\_potencia}$ corresponde a la frecuencia donde se alcanza el 99% de la potencia acumulada, y $f_\text{min\_potencia}$ corresponde al 1% si se considera el límite inferior.

Finalmente, la función grafica la PSD de la señal, mostrando el ancho de banda con líneas verticales punteadas y etiquetando su valor. Esta visualización permite interpretar qué rangos de frecuencia concentran la mayor parte de la energía de la señal, información fundamental tanto para señales biomédicas como para audio.

## Conclusión
En este trabajo se aplicó el método de Welch para estimar la densidad espectral de potencia (PSD) de diferentes tipos de señales: biomédicas (ECG y PPG) y acústicas (audio), además de analizar el efecto del ruido sobre el ECG. Esta metodología permitió no solo visualizar la distribución de energía en el dominio frecuencial, sino también cuantificar el ancho de banda efectivo de cada señal mediante un criterio basado en el 99 % de la potencia acumulada.

Los resultados obtenidos muestran diferencias claras entre las señales: el ECG presentó un ancho de banda reducido (aproximadamente 30 Hz), con su energía concentrada en bajas frecuencias, lo que concuerda con la naturaleza fisiológica de la actividad cardíaca. La PPG mostró un ancho de banda aún más estrecho (alrededor de 5 Hz), reflejando la suavidad de la señal asociada al pulso sanguíneo. La señal de audio evidenció un ancho de banda significativamente mayor (aproximadamente 1035 Hz), propio de señales complejas ricas en armónicos. Finalmente, el análisis del ECG con ruido demostró que, aunque el nivel espectral aumenta en todo el rango de frecuencias, el ancho de banda estimado se mantiene relativamente estable, evidenciando la robustez del método de Welch frente a perturbaciones moderadas.

## Autoevaluación del aprendizaje
A lo largo de este trabajo, logramos consolidar y aplicar varios conceptos fundamentales del análisis de señales en el dominio de la frecuencia. Comprendimos en profundidad cómo funciona el método de Welch para estimar la densidad espectral de potencia y cómo la segmentación, el solapamiento y la ventana elegida influyen directamente en la resolución y suavizado del espectro.
